# Simple GPROF rain rate retrieval

In this notebook we will apply QRNNs to retrieve rain rates from passive microwave observations from the Global Precipitation Measurement (GPM) mission. We will then use the model to classify pixels based on their rain rate and compare the performance of the classifier to that of the current algorithm. 

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from quantnn.examples.gprof_simple import GPROFDataset, download_data
download_data()

## The Data

The input data consists of single-pixel observations from the GMI radiometer combined with the surface type, column-integrated water vapor and two-meter temperature. Expanding the 14 surface types to one-hot encoding results in 29 input features.

The output data consists of surface rain rates, which are, for the largest part, derived from GMI observations combined with the precipitation radar, which is flown together with GMI on board the GPM Core Observatory satellite. The combined radar-radiometer observations improve the accuracy of the rain retrieval, which is why this data is suitable to be used as ground truth.

In [2]:
training_data = GPROFDataset("data/training_data_gmi_small.nc", batch_size=128)
validation_data = GPROFDataset("data/training_data_gmi_small.nc", batch_size=128, normalizer=training_data.normalizer)

In [3]:
from torch.utils.data import DataLoader
training_loader = DataLoader(training_data, batch_size=None, num_workers=2)
validation_loader = DataLoader(validation_data, batch_size=None, num_workers=2)

## Defining a QRNN model

In [4]:
import quantnn as qn
quantiles = [0.01, 0.05, 0.15, 0.25, 0.35, 0.45, 0.5, 0.55, 0.65, 0.75, 0.85, 0.95, 0.99]
qrnn = qn.QRNN(n_inputs=29, quantiles=quantiles, model=(4, 256, "relu"))

## Training the QRNN

In [5]:
import torch
n_epochs = 10
optimizer = torch.optim.SGD(qrnn.model.parameters(), lr=0.1, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)
qrnn.train(training_loader,
           validation_loader,
           optimizer=optimizer,
           scheduler=scheduler,
           n_epochs=n_epochs,
           device="gpu")
optimizer = torch.optim.SGD(qrnn.model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)
qrnn.train(training_loader,
           validation_loader,
           optimizer=optimizer,
           scheduler=scheduler,
           n_epochs=n_epochs,
           device="gpu")
optimizer = torch.optim.SGD(qrnn.model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)
qrnn.train(training_loader,
           validation_loader,
           optimizer=optimizer,
           scheduler=scheduler,
           n_epochs=n_epochs,
           device="gpu")

Output()

                                                          
                                                          
                     Training history                     
                                                          
                                                          
       Epoch         Training loss      Validation loss   
 ──────────────────────────────────────────────────────── 
     #      LR           Total               Total        
                                                          
      1   0.1000         0.020               0.018        
      2   0.0976         0.018               0.018        
      3   0.0905         0.017               0.017        
      4   0.0794         0.017               0.016        
      5   0.0655         0.016               0.016        
      6   0.0500         0.016               0.015        
      7   0.0345         0.015               0.015        
      8   0.0206         0.015               0.015        
      9   0.0095         0.015               0.015        
     10   0.0024         0.015               0.014

Output()

                                                          
                                                          
                     Training history                     
                                                          
                                                          
       Epoch         Training loss      Validation loss   
 ──────────────────────────────────────────────────────── 
     #      LR           Total               Total        
                                                          
      1   0.0100         0.015               0.015        
      2   0.0098         0.015               0.015        
      3   0.0090         0.015               0.015        
      4   0.0079         0.015               0.014        
      5   0.0065         0.014               0.014        
      6   0.0050         0.014               0.014        
      7   0.0035         0.014               0.014        
      8   0.0021         0.014               0.014        
      9   0.0010         0.014               0.014        
     10   0.0002         0.014               0.014

Output()

                                                          
                                                          
                     Training history                     
                                                          
                                                          
       Epoch         Training loss      Validation loss   
 ──────────────────────────────────────────────────────── 
     #      LR           Total               Total        
                                                          
      1   0.0010         0.014               0.014        
      2   0.0010         0.014               0.014        
      3   0.0009         0.014               0.014        
      4   0.0008         0.014               0.014        
      5   0.0007         0.014               0.014        
      6   0.0005         0.014               0.014        
      7   0.0003         0.014               0.014        
      8   0.0002         0.014               0.014        
      9   0.0001         0.014               0.014        
     10   0.0000         0.014               0.014

## Classifying raining pixels

We will test our retrieval by using it to predict whether the rain rate at a given pixel is larger than a threshold of $0.01\ \text{mm/h}$. The estimated probability of a given pixel being larger than this value is also part of the GPROF retrieval output and we will use this output to evaluate the QRNN retrieval.

In [6]:
import quantnn as q
from quantnn.examples.gprof_rr import GPROFTestset
test_data = GPROFTestset("data/test_data_gmi_small.nc", normalizer=training_data.normalizer)
p_qrnn = qrnn.probability_larger_than(test_data.x, 0.01).detach().numpy()
p_gprof = test_data.y_pop / 100.0
truth = test_data.y.ravel() > 0.01

ModuleNotFoundError: No module named 'quantnn.examples.gprof_rr'

In [ ]:
def precision_and_recall_curve(p_pred, truth, n=101):
    ps = np.linspace(0, 1, n)
    precision = np.zeros(n)
    recall = np.zeros(n)
    for i, p in enumerate(ps):
        predicted = p_pred > p
        precision[i] = np.sum(predicted * truth) / np.sum(predicted)
        recall[i] = np.sum(predicted * truth) / np.sum(truth)
    return precision, recall

precision_qrnn, recall_qrnn = precision_and_recall_curve(p_qrnn, truth)
precision_gprof, recall_gprof = precision_and_recall_curve(p_gprof, truth)

In [ ]:
from quantnn.plotting import set_style
set_style()
f, ax = plt.subplots(1, 1)
ax.plot(recall_qrnn, precision_qrnn, label="QRNN")
ax.plot(recall_gprof, precision_gprof, label="GPROF")
ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
ax.legend()